In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import xgboost as xgb
from sklearn.metrics import accuracy_score, recall_score, precision_score, roc_auc_score, balanced_accuracy_score, ConfusionMatrixDisplay
import warnings
warnings.filterwarnings('ignore')
import os
import gc
import joblib
import pandas as pd
import numpy as np
from sklearn import metrics, preprocessing
from tensorflow.keras import layers
from tensorflow.keras import optimizers
from tensorflow.keras.models import Model, load_model
from tensorflow.keras import callbacks
from tensorflow.keras import backend as K
from tensorflow.keras import utils
from sklearn import model_selection

In [34]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, BatchNormalization

In [35]:
from numpy.random import seed
seed(42)

In [36]:
#Import the Data Set 
df =  pd.read_csv('../Data/df_clean.csv')

In [37]:
#Binarize the Predictor Variable: Satisfaction Rating 
# 1: Not Satisfied with Rental (Satisfaction Rating of 1,2,3) 
# 0: Satisfied with Rental (Satisfaction Rating 4,5)
df['Rating'] = np.where(df['Rating'] <= 3, 1, 0)

In [38]:
from sklearn.model_selection import train_test_split

In [39]:
#Drop Unnecessary Columns 
df.drop(columns=["Brand","Dress_Description","Details","Product_details","Retail_price"],inplace=True)
df.dropna(inplace=True)

In [40]:
num_cols = ["Height","Age","Weight","Rent_price","Number_of_reviews","BMI"]

In [41]:
features = [c for c in df.columns if c not in num_cols and c not in "Rating"]

In [42]:
for col in features:
        df.loc[:,col] = df[col].astype(str) 

In [43]:
for feat in features:
        lbl_enc = preprocessing.LabelEncoder()
        df.loc[:,feat]=lbl_enc.fit_transform(df[feat].values)

In [44]:
df.head()

,Label,Size,Overall_fit,Rented_for,Size_usually_worn,Height,Age,Bust_size,Body_type,Weight,Rating,Date,Rent_price,Number_of_reviews,Sleeves,Neckline,Dress_Style,BMI
0,0,95,2,4,12,66,31,39,1,142,0,1,70,33,6,9,6,22.916896
1,0,91,0,5,11,67,33,51,3,150,0,1,70,33,6,9,6,23.490755
2,0,95,0,1,11,66,27,38,4,140,0,1,70,33,6,9,6,22.594123
3,0,88,2,7,2,66,42,50,4,181,0,1,70,33,6,9,6,29.210973
4,0,88,2,1,2,70,48,40,4,165,0,1,70,33,6,9,6,23.672449


In [45]:
df.columns

Index(['Label', 'Size', 'Overall_fit', 'Rented_for', 'Size_usually_worn',
       'Height', 'Age', 'Bust_size', 'Body_type', 'Weight', 'Rating', 'Date',
       'Rent_price', 'Number_of_reviews', 'Sleeves', 'Neckline', 'Dress_Style',
       'BMI'],
      dtype='object')

In [46]:
X = df[['Label', 'Size', 'Overall_fit', 'Rented_for',
        'Size_usually_worn','Height', 'Age', 'Bust_size',
        'Body_type', 'Weight',  'Date','Rent_price', 
        'Number_of_reviews', 'Sleeves', 'Neckline', 'Dress_Style',
       'BMI']]

y=df[["Rating"]]

In [47]:
X = np.asarray(X)

In [48]:
y = np.asarray(y)

In [49]:
X_train, X_test, y_train, y_test = train_test_split(
  X,
  y,
  test_size=0.20,random_state=999, stratify=y
)

In [50]:
from tensorflow.keras.initializers import RandomNormal, Constant

In [54]:
model = Sequential([
    Dense(64, input_shape=(17,), activation="relu"),
    BatchNormalization(),
    Dense(128, activation='relu'),
    BatchNormalization(),
    Dense(128, activation='relu'),
    BatchNormalization(),
    Dense(64, activation='relu'),
    BatchNormalization(),
    Dense(64, activation='relu'),
    BatchNormalization(
        momentum=0.95, 
        epsilon=0.005,
        beta_initializer=RandomNormal(mean=0.0, stddev=0.05), 
        gamma_initializer=Constant(value=0.9)
    ),
    Dense(1, activation='softmax')
]);

In [55]:
model.compile(
    optimizer='adam', 
    loss='categorical_crossentropy', 
    metrics=['accuracy']
)

In [56]:
history = model.fit(
    X_train, 
    y_train, 
    epochs=200, 
    validation_split=0.25, 
    batch_size=40, 
    verbose=2
)

Epoch 1/200
2420/2420 - 5s - loss: 0.0000e+00 - accuracy: 0.1665 - val_loss: 0.0000e+00 - val_accuracy: 0.1663
Epoch 2/200
2420/2420 - 3s - loss: 0.0000e+00 - accuracy: 0.1665 - val_loss: 0.0000e+00 - val_accuracy: 0.1663
Epoch 3/200
2420/2420 - 3s - loss: 0.0000e+00 - accuracy: 0.1665 - val_loss: 0.0000e+00 - val_accuracy: 0.1663
Epoch 4/200
2420/2420 - 3s - loss: 0.0000e+00 - accuracy: 0.1665 - val_loss: 0.0000e+00 - val_accuracy: 0.1663
Epoch 5/200
2420/2420 - 3s - loss: 0.0000e+00 - accuracy: 0.1665 - val_loss: 0.0000e+00 - val_accuracy: 0.1663
Epoch 6/200
2420/2420 - 3s - loss: 0.0000e+00 - accuracy: 0.1665 - val_loss: 0.0000e+00 - val_accuracy: 0.1663
Epoch 7/200
2420/2420 - 3s - loss: 0.0000e+00 - accuracy: 0.1665 - val_loss: 0.0000e+00 - val_accuracy: 0.1663
Epoch 8/200
2420/2420 - 4s - loss: 0.0000e+00 - accuracy: 0.1665 - val_loss: 0.0000e+00 - val_accuracy: 0.1663
Epoch 9/200
2420/2420 - 4s - loss: 0.0000e+00 - accuracy: 0.1665 - val_loss: 0.0000e+00 - val_accuracy: 0.1663
E

Epoch 75/200
2420/2420 - 3s - loss: 0.0000e+00 - accuracy: 0.1665 - val_loss: 0.0000e+00 - val_accuracy: 0.1663
Epoch 76/200
2420/2420 - 3s - loss: 0.0000e+00 - accuracy: 0.1665 - val_loss: 0.0000e+00 - val_accuracy: 0.1663
Epoch 77/200
2420/2420 - 3s - loss: 0.0000e+00 - accuracy: 0.1665 - val_loss: 0.0000e+00 - val_accuracy: 0.1663
Epoch 78/200
2420/2420 - 3s - loss: 0.0000e+00 - accuracy: 0.1665 - val_loss: 0.0000e+00 - val_accuracy: 0.1663
Epoch 79/200
2420/2420 - 3s - loss: 0.0000e+00 - accuracy: 0.1665 - val_loss: 0.0000e+00 - val_accuracy: 0.1663
Epoch 80/200
2420/2420 - 3s - loss: 0.0000e+00 - accuracy: 0.1665 - val_loss: 0.0000e+00 - val_accuracy: 0.1663
Epoch 81/200
2420/2420 - 3s - loss: 0.0000e+00 - accuracy: 0.1665 - val_loss: 0.0000e+00 - val_accuracy: 0.1663
Epoch 82/200
2420/2420 - 3s - loss: 0.0000e+00 - accuracy: 0.1665 - val_loss: 0.0000e+00 - val_accuracy: 0.1663
Epoch 83/200
2420/2420 - 3s - loss: 0.0000e+00 - accuracy: 0.1665 - val_loss: 0.0000e+00 - val_accuracy:

Epoch 148/200
2420/2420 - 3s - loss: 0.0000e+00 - accuracy: 0.1665 - val_loss: 0.0000e+00 - val_accuracy: 0.1663
Epoch 149/200
2420/2420 - 3s - loss: 0.0000e+00 - accuracy: 0.1665 - val_loss: 0.0000e+00 - val_accuracy: 0.1663
Epoch 150/200
2420/2420 - 3s - loss: 0.0000e+00 - accuracy: 0.1665 - val_loss: 0.0000e+00 - val_accuracy: 0.1663
Epoch 151/200
2420/2420 - 4s - loss: 0.0000e+00 - accuracy: 0.1665 - val_loss: 0.0000e+00 - val_accuracy: 0.1663
Epoch 152/200
2420/2420 - 4s - loss: 0.0000e+00 - accuracy: 0.1665 - val_loss: 0.0000e+00 - val_accuracy: 0.1663
Epoch 153/200
2420/2420 - 3s - loss: 0.0000e+00 - accuracy: 0.1665 - val_loss: 0.0000e+00 - val_accuracy: 0.1663
Epoch 154/200
2420/2420 - 4s - loss: 0.0000e+00 - accuracy: 0.1665 - val_loss: 0.0000e+00 - val_accuracy: 0.1663
Epoch 155/200
2420/2420 - 3s - loss: 0.0000e+00 - accuracy: 0.1665 - val_loss: 0.0000e+00 - val_accuracy: 0.1663
Epoch 156/200
2420/2420 - 3s - loss: 0.0000e+00 - accuracy: 0.1665 - val_loss: 0.0000e+00 - val_